# Set up environment

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import tensorflow.keras.utils as utils

import os
import PIL.Image as Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from usda import UsdaClient

**Declare parameters**

In [2]:
ROOT_DIR = os.path.dirname(os.getcwd())

IMAGE_SIZE = 512

API_KEY = "4INghUtThsIBWPTIcvfKyf0kNS6MtSXcC4R6mpNB"

**Enable GPU**

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Running on", device)

Running on cuda:0


# Input model

**Fetch model architecture**

In [4]:
print("Fetching model architecture... ", end="")

backbone = torchvision.models.wide_resnet101_2(pretrained=False)

# remove classification head
backbone = torch.nn.Sequential(*list(backbone.children())[:-2])
backbone.out_channels = 2048

model = torchvision.models.detection.FasterRCNN(backbone, num_classes=256)

model = model.to(device)

print("done")

Fetching model architecture... done


**Load model weights**

In [5]:
MODEL_PATH = os.path.join(ROOT_DIR, "models", "food_detection", "condensed", "mobilenet_backbone.pt")

print("Loading learnt model weights...", end="")
model.load_state_dict(torch.load(MODEL_PATH))
print("done")

Loading learnt model weights...

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\OneDrive - Durham University\\Year 3\\Project\\Im2Calories\\notebooks\\models\\food_detection\\wideres_backbone.pt'

# Predict

**Fetch class names**

In [ ]:
FOOD256_DIR = os.path.join(os.path.abspath(os.sep), "Datasets", "food256")

class_names = []

with open(os.path.join(FOOD256_DIR, "category.txt")) as file:
    file.readline()
    for line in file.readlines():
        class_names.append(line.split('\t')[1].strip())
                           
class_names

**Load image from url**

In [ ]:
def loadImage(url):
    filename = url.split('/')[-1]
    img = utils.get_file(filename, url)
    img = Image.open(img)
    return img

img = loadImage("https://ichef.bbci.co.uk/food/ic/food_16x9_832/recipes/fivespicespareribs_70976_16x9.jpg")
img

**Format image for model input**

In [ ]:
# custom transform for R-CNN input
class CustomTransform:
    def __init__(self, image_size):
        self.image_size = image_size
        
    def __call__(self, img):
        # add padding
        w, h = img.size
        xPad = IMAGE_SIZE - w
        yPad = IMAGE_SIZE - h
        img = transforms.functional.pad(img, (0, 0, xPad, yPad))

        # convert to tensor
        img = transforms.functional.to_tensor(img)

        # normalize
        img = img / 255
        return img

transform = CustomTransform(image_size=IMAGE_SIZE)

img = transform(img)
images = img.unsqueeze(0)

**Apply model on image**

In [ ]:
def predict(images):
    images = images.to(device)
    model.eval()
    
    with torch.no_grad():
        outputs = model(images)
    
    return outputs

results = predict(images)
results

**Draw predicted bounding boxes on image**

In [ ]:
CONFIDENCE_THRESHOLD = 0.4

def drawResults(img, target):
    fig, axis= plt.subplots(1)

    img = np.transpose(img, (1,2,0))
    img = img * 255
    img = np.clip(img, 0, 1)
    plt.imshow(img)

    # draw results
    for i in range(len(target["labels"])):
        if(target["scores"][i] < CONFIDENCE_THRESHOLD):
            break
        
        # draw bounding box
        x1, y1, x2, y2 = target["boxes"][i]
        box = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, edgecolor='r', facecolor='none')
        axis.add_patch(box)
        axis.text(x1, y2, class_names[target["labels"][i]], fontdict=dict(color='w', weight='bold'), bbox=dict(facecolor='r', edgecolor='none'))
        foods.append(class_names[target["labels"][i]])
        
    plt.show()

foods = []
drawResults(img, results[0])

# Nutritional lookup

**Search USDA for foods with predicted class**

In [ ]:
client = UsdaClient(API_KEY)

nutrients = {}

for food_item in foods:
    # seach USDA with predicted image class as search term
    food_results = client.search_foods(food_item, 1)

    food = next(food_results)

    # fetch food report for top result
    food_report = client.get_food_report(food.id)
    
    # add nutritional information to dictionary
    for nutrient in food_report.nutrients:
        key = "{0} ({1})".format(nutrient.name, nutrient.unit)
        nutrients[key] = nutrients.get(key, 0) + nutrient.value
    
nutrients